## Getting 10,000 urls from DMCA-COX
-  ssh -N dmca@cox.media.mit.edu -L 3310:cox.media.mit.edu:3306
```
# import pandas as pd
# from sqlalchemy import create_engine
# constr= 'mysql://civilservant@localhost:3310/civilservant_production'
# con = create_engine(constr)
# con.execute('show tables;').fetchall()
```

In [14]:
from utils.common import DbEngine
import os
import inspect
from app.models import TwitterStatus
import json
from sqlalchemy.sql.expression import func as sqlfunc
from functools import reduce
from url_unshortener import bulkUnshorten

BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))), "..")
ENV = 'development'
db_session = DbEngine(os.path.join(BASE_DIR, "config") + "/{env}.json".format(env=ENV)).new_session()

STATUSES_TO_GET = 10000
json_fname = f'../tests/fixture_data/twitter_statuses_{STATUSES_TO_GET}.json'
try:
    status_urls_flat = json.load(open(json_fname,'r'))
except FileNotFoundError:
    def get_url_part(status):
        try:
            status_data = json.loads(status.status_data)
            return status_data['entities']['urls']
        except TypeError:
            # print(status.id)
            return []
        
    statuses = db_session.query(TwitterStatus).order_by(sqlfunc.rand()).limit(STATUSES_TO_GET).all()
    assert len(statuses) == STATUSES_TO_GET
        
    status_urls = [get_url_part(status) for status in statuses]
    
    status_urls_flat = []
    for status_url in status_urls:
        for url_dict in status_url:
                # print(url_dict)
                status_urls_flat.append(url_dict['url'])
    json.dump(status_urls_flat, open(json_fname,'w'))

status_urls_flat[:5]
## proportion of statuses having urls
len(status_urls_flat),  STATUSES_TO_GET

(2761, 10000)

In [2]:
%timeit -r 1 -n 1 bulkUnshorten(status_urls_flat[:10])

4.28 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
%timeit -r 1 -n 1 bulkUnshorten(status_urls_flat[:100])

8.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
%timeit -r 1 -n 1 bulkUnshorten(status_urls_flat[:1000])

1min 33s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%timeit -r 1 -n 1 bulkUnshorten(status_urls_flat[:10000])

2min 21s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
db_session.query(TwitterStatus).count()

473243

In [13]:
total_tweets = 38000000
tweet_url_prop = 0.27
urls_per_second = 10 
urls_per_minute = 650
urls_per_hour = 38000

total_tweets * tweet_url_prop / urls_per_hour

270.0